In [1]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re

BASE_URL = 'https://www.thequint.com/'
country = 'bangladesh'
initial_url = f'{BASE_URL}search?q={country}'

driver = webdriver.Chrome()
driver.get(initial_url)

time.sleep(10)

#COLLECTING ALL NEWS LINKS

load_more_button = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div/div[2]/div[4]/button')

driver.execute_script("arguments[0].scrollIntoView();", load_more_button)

number_of_clicks = 2

while number_of_clicks > 0:
    
    if load_more_button:
        
        time.sleep(5)
        
        ActionChains(driver)\
        .click(load_more_button)\
        .perform()

        time.sleep(5)

        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        
    else:
        
        break 
    
    number_of_clicks -=1
    

soup = BeautifulSoup(driver.page_source, 'html.parser')

news_unordered_list = soup.find('ul', class_ = '_11ALQ')

all_link_divs = news_unordered_list.find_all('div', class_ = '_3YCSF')

all_news_links = []

for each_div in all_link_divs:
    
    all_news_links.append(each_div.find('a').get('href'))
    

driver.quit()

#SCRAPING ALL NEWS LINKS

data_list = []
counter = 0

#SCRAPING THE FIRST 15 links

for url in all_news_links[:15]:
    
    if not 'photos' in url:
    
        response = requests.get(url)

        soup = BeautifulSoup(response.text, 'html.parser')

        #TITLE

        title_tag = soup.find('h1', class_ = 'ieh5X RXXC0')
        title = title_tag.text if title_tag else 'Title not found'
        title_translation = 'None'

        #CONTENT SUMMARY

        content_summary_tag = soup.find('h2', class_ = '_2QRqw')
        content_summary = content_summary_tag.text if content_summary_tag else 'Content Summary not found'
        summary_translation = 'None'

        #AUTHOR

        author_tag = soup.find('div', class_ = '_1zOzV')
        author = author_tag.find('a').text if author_tag else 'Author not found'

        #Date
        date_info = soup.find('time', class_ = '_1yYGD')

        if date_info:

            date_data = date_info.get('datetime')

            only_date = date_data.split('T')[0]
            only_time = date_data.split('T')[1]
            time = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
            cleaned_date = f"{only_date},{time}"

            source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M") + timedelta(hours = 5, minutes = 30) 
            bangladesh_localtime = source_localtime + timedelta(minutes=30)

        else:

            date_data = 'Date data not found'


        content = []

        main_divs = soup.find_all('div', class_ = 'story-element story-element-text')
        
        if main_divs:

            for each_div in main_divs:

                all_paras = each_div.find_all('p') or each_div.find_all('ul') or soup.find_all('blockquote')

                for each_para in all_paras:

                    content.append(each_para.text)

            full_content = ''.join(content)
            
            temp_content = full_content.split('(At The Quint,')
            
            full_content = temp_content[0]

        else:

            full_content = 'Content not found'

        content_translation = 'None'


        data_dict = {
                "url": url,
                "title": title,
                "content": full_content,
                "content_summary": content_summary,
                "title_translation":title_translation,
                "content_translation":content_translation,
                "summary translation":summary_translation,
                "author": author,
                "country": country,
                'source_localtime': source_localtime,
                'bangladesh_localtime': bangladesh_localtime

            }

        counter+=1


        if (full_content != "Content not found" or title != 'Title not found'):

                    if data_dict not in data_list:
                            # Adding to data list
                            data_list.append(data_dict)
                            print(f'Link {counter} added')
        else:
                    print(f'Link {counter}')
                    print('Skipped due to missing info.')


df = pd.DataFrame(data_list)
df.head()

csv_filename = f"{country}_The_Quint.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["bangladesh_localtime"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="date", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)


Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added
Link 11 added
Link 12 added
Link 13 added


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://www.thequint.com/news/webqoof/false-co...,Murder of a Hindu Family in Bangladesh Given F...,A set of four graphic images of three corpses ...,The accused was identified as Rajiv Kumar Bhow...,None,None,None,Khushi Mehrotra,bangladesh,2024-02-05 17:12:00,2024-02-05 17:42:00
1,https://www.thequint.com/sports/cricket/india-...,India Slip Below Bangladesh in Latest WTC Poin...,India slipped down the World Test Championship...,India are currently placed 5th in the 2023-25 ...,None,None,None,IANS,bangladesh,2024-01-29 17:25:00,2024-01-29 17:55:00
2,https://www.thequint.com/news/world/padma-shri...,"Renowned Bangladesh Artist, 4 French Citizens ...","On the eve of India's 75th Republic Day, the I...",Foxconn Chairman Young Liu has also been confe...,None,None,None,The Quint,bangladesh,2024-01-26 14:42:00,2024-01-26 15:12:00
3,https://www.thequint.com/sports/cricket/u-19-w...,U-19 World Cup: Uday Saharan-Led India Registe...,Team India registered a massive 84-run triumph...,#IndvsBan | India thumped Bangladesh by 84 run...,None,None,None,The Quint,bangladesh,2024-01-20 22:42:00,2024-01-20 23:12:00
4,https://www.thequint.com/sports/cricket/india-...,India U19 vs Bangladesh U19: When and Where To...,IND vs BAN U19 Cricket World Cup 2024: U-19 te...,India vs Bangladesh U-19 World Cup 2024: Know ...,None,None,None,Saima Andrabi,bangladesh,2024-01-20 11:41:00,2024-01-20 12:11:00
